In [1]:
import hopsworks
import time
from datetime import datetime
from datetime import timedelta

project = hopsworks.login()
fs = project.get_feature_store()

2025-10-09 20:47:48,949 INFO: Python Engine initialized.

Logged in to project, explore it here https://snurran.devnet.hops.works/p/4217


### Get some sample user_ids for online inference tests
Read them up from `user_clicks`

In [2]:
num_samples = 100

clicks_fg = fs.get_feature_group("user_clicks")
df = clicks_fg.read()

max_event_time = df['event_time'].max() 
max_event_time_dt = datetime.fromtimestamp(max_event_time)

one_min = int((max_event_time_dt - timedelta(minutes=1)).timestamp())
ten_min = int((max_event_time_dt - timedelta(minutes=10)).timestamp())
thirty_min = int((max_event_time_dt - timedelta(minutes=30)).timestamp())
one_hr = int((max_event_time_dt - timedelta(hours=1)).timestamp())

sampled_user_ids = df.sample(n=num_samples, random_state=42)

2025-10-09 20:47:49,221 WARNING: VersionWarning: No version provided for getting feature group `user_clicks`, defaulting to `1`.

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.23s) 


### Use feature view to retrieve feature vectors

In [7]:
fv = fs.get_feature_view("precomputed_click_counts")
start = time.perf_counter()

for idx, row in sampled_user_ids.iterrows():
    user_id = row['user_id']
    r = fv.get_feature_vector(entry={"user_id": user_id})
    click_count_1_min = r[2]
    click_count_10_min = r[3]
    click_count_30_min = r[4]
    click_count_1_hour = r[5]
    print(
        f"user_id={user_id}, "
        f"click_count_1_min={click_count_1_min}, "
        f"click_count_10_min={click_count_10_min}, "
        f"click_count_30_min={click_count_30_min}, "
        f"click_count_1_hour={click_count_1_hour}"
    )
        
end = time.perf_counter()
print(f"Time taken (s): {(end-start)}")

2025-10-09 20:53:48,487 WARNING: VersionWarning: No version provided for getting feature view `precomputed_click_counts`, defaulting to `1`.

2025-10-09 20:53:49,480 WARNING: VersionWarning: No training dataset version was provided to initialise serving. Defaulting to version 1.

user_id=44, click_count_1_min=17, click_count_10_min=154, click_count_30_min=448, click_count_1_hour=643
user_id=8, click_count_1_min=13, click_count_10_min=159, click_count_30_min=455, click_count_1_hour=501
user_id=47, click_count_1_min=13, click_count_10_min=141, click_count_30_min=306, click_count_1_hour=306
user_id=82, click_count_1_min=14, click_count_10_min=145, click_count_30_min=430, click_count_1_hour=774
user_id=76, click_count_1_min=13, click_count_10_min=131, click_count_30_min=355, click_count_1_hour=355
user_id=90, click_count_1_min=15, click_count_10_min=148, click_count_30_min=363, click_count_1_hour=363
user_id=4, click_count_1_min=17, click_count_10_min=156, click_count_30_min=462, click_cou

### Benchmark with MySQL Client
For extra performance, we can go directly to the MySQL Server.

In [3]:
import pymysql

host = "onlinefs.mysql.service.consul"
port = 3306
database = "aggregations"
user = "aggregations_jim00000"
password = "tRwvOWqSFNUOaIXzXyaIzDIfIKxjQSzG"

conn = pymysql.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database,
    ssl=None  
)

start = time.perf_counter()
total_count = 0
for idx, row in sampled_user_ids.iterrows():
    user_id = row['user_id']
    cursor = conn.cursor()
    query = "SELECT * FROM precomputed_click_counts_1 WHERE user_id = %s"
    cursor.execute(query, (user_id,))
    r = cursor.fetchone()
    cursor.close()
    click_count_1_min = r[2]
    click_count_10_min = r[3]
    click_count_30_min = r[4]
    click_count_1_hour = r[5]
    total_count += 1
    print(
        f"user_id={user_id}, "
        f"click_count_1_min={click_count_1_min}, "
        f"click_count_10_min={click_count_10_min}, "
        f"click_count_30_min={click_count_30_min}, "
        f"click_count_1_hour={click_count_1_hour}"
    )
end = time.perf_counter()
print(f"Time taken (s): {(end-start)}")
print(f"Total Count: {total_count}")

conn.close()


user_id=44, click_count_1_min=17, click_count_10_min=154, click_count_30_min=448, click_count_1_hour=643
user_id=8, click_count_1_min=13, click_count_10_min=159, click_count_30_min=455, click_count_1_hour=501
user_id=47, click_count_1_min=13, click_count_10_min=141, click_count_30_min=306, click_count_1_hour=306
user_id=82, click_count_1_min=14, click_count_10_min=145, click_count_30_min=430, click_count_1_hour=774
user_id=76, click_count_1_min=13, click_count_10_min=131, click_count_30_min=355, click_count_1_hour=355
user_id=90, click_count_1_min=15, click_count_10_min=148, click_count_30_min=363, click_count_1_hour=363
user_id=4, click_count_1_min=17, click_count_10_min=156, click_count_30_min=462, click_count_1_hour=470
user_id=49, click_count_1_min=16, click_count_10_min=143, click_count_30_min=411, click_count_1_hour=754
user_id=11, click_count_1_min=18, click_count_10_min=145, click_count_30_min=446, click_count_1_hour=530
user_id=60, click_count_1_min=13, click_count_10_min=154,